In [1]:
import openai
import tiktoken
import pandas as pd
import os
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [2]:
import api 
gpt = api.GPT(model='gpt-3.5-turbo')

In [3]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True) # ROUGE-L measures the longest common subsequence (LCS) between the candidate text and the reference text.

candidate_summary = "the cat was found under the bed"
reference_summary = "the cat was under the bed"
scores = scorer.score(reference_summary, candidate_summary)
print(scores)

# BLEU score measures the similarity between the candidate text and the reference text using n-grams, which are contiguous sequences of n words
print(sentence_bleu([reference_summary.split()], candidate_summary.split()))

#for key in scores:
    #print(f'{key}: {scores[key]}')

{'rougeL': Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)}
8.44484326442819e-78


/Users/claesia/hiring_task_env/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [4]:
def gpt_chat(model, system_context, prompt, context_len):
    
    messages=[{"role":"system","content": system_context},
          {"role":"user","content": prompt}]

    token_count = num_tokens_from_string(messages)
    responses = chat(model, messages, max_tokens = context_len-token_count) 
    
    return responses

In [5]:
system_context_pt = """
Você é um professor e sua função envolve a criação de materiais educacionais e avaliações para seus alunos. 
"""

prompt_sub_pt = """
Prepare um teste com 10 perguntas subjetivas com base no texto fornecido abaixo. 
Não ofereça opiniões ou análises; use apenas as informações solicitadas fornecidas. 
Não forneça o gabarito.
Suas perguntas deve seguir este modelo: (título do teste)\n Pergunta 1: (pergunta gerada)\n etc.
"""

prompt_obj_pt = """
Prepare um teste com 10 perguntas com base no texto fornecido abaixo. 
Não ofereça opiniões ou análises; use apenas as informações solicitadas fornecidas. 
Não forneça o gabarito.
Suas perguntas deve seguir este modelo: (título do teste)\n Pergunta 1: (pergunta gerada)\n etc.
"""

system_context_en = """
You are a teacher and your role involves creating educational materials and assessments for your students. 
"""

prompt_obj_en = """
Prepare a test with 10 questions based on the text provided below.
Do not offer opinions or analysis; use only the requested information provided.
Do not provide the answer key.
Your response should follow this template: (test title)\n Question 1: (question generated)\n etc. 
"""
prompt_sub_en = """
Prepare a test with 10 subjective questions based on the text provided below.
Do not offer opinions or analysis; use only the requested information provided.
Do not provide the answer key.
Your response should follow this template: (test title)\n Question 1: (question generated)\n etc. 
"""

In [6]:
system_context_stud_pt = """
Você é um estudante e seu professor pediu que você complete um teste com 10 perguntas. 
Seu objetivo é responder a essas perguntas da melhor forma possível, demonstrando sua compreensão do tópico.
"""

prompt_stud_pt = """
O teste a seguir consiste em 10 perguntas.
Por favor, responda às perguntas a seguir. 
Suas respostas devem ser concisas e precisas e devem seguir este modelo: (título do teste)\n Pergunta 1: (contexto da pergunta) \n Sua resposta: \n\n etc. \n```\n 
"""

system_context_stud_en = """
You are a student and your teacher has asked you to complete a test that consists of 10 questions. 
Your goal is to answer these questions to the best of your ability, demonstrating your understanding of the topic.
"""

prompt_stud_en = """
The below is a test consisting of 10 questions.
Please answer the following questions.
Your responses should be concise and accurate and should follow this template: (test title)\n Question 1: (question context) \n Your answer: \n\n etc. \n```\n 
"""

In [7]:
add_pt = "\n Use o texto acima como base para responder o teste."
add_en = "\n Use the text below as a basis to answer the test."

In [8]:
texto = """
\n\nO ano de 1492 foi marcado pelo choque entre dois mundos diferentes, de um lado, a cultura europeia, e de outro, a cultura indígena. Trata-se do início da colonização da América, promovida, principalmente, por portugueses e espanhóis em suas buscas por riquezas e conquistas. Outra data de caráter simbólico importante para a história é a de 1494, quando foi assinado o Tratado de Tordesilhas.
O Tratado de Tordesilhas foi um acordo político entre os países ibéricos, Portugal e Espanha, para oficializar a divisão do novo continente encontrado. O tratado definia como linha imaginária uma demarcação no meridiano localizado a 370 léguas a oeste da ilha de Cabo Verde. As terras que se localizavam a leste pertenceriam aos portugueses e as terras a oeste ficariam com os espanhóis. 
Em 1500, os navios portugueses ancoraram no litoral brasileiro, dando início ao que os historiadores chamaram de América Portuguesa. O período correspondente aos trinta primeiros anos (1500-1530) das expedições portuguesas no Brasil foi denominado de Período Pré-Colonial, ou seja, foi um momento em que Portugal ainda estava investigando as riquezas desse “novo mundo”.
Após esse momento, na expedição sob a liderança de Martin Afonso de Souza, em 1530, os portugueses resolveram oficializar a colonização brasileira por receio de perder o território para outros países europeus e pela perspectiva de descobrir riquezas e obter lucros. Uma das maneiras de assegurar o território brasileiro era povoando-o, para isso, o rei de Portugal, em 1534, criou as chamadas capitanias hereditárias. Essas capitanias representavam largas faixas de terras sob o poder dos capitães donatários que tinham o objetivo de distribui-las para gerar povoamentos e, claro, lucros à coroa.
Porém, esse tipo de governo com um capitão donatário para cada faixa de terra não deu certo e o controle da colônia ficou a cargo de Tomé de Souza, o primeiro governador-geral do Brasil, a partir de 1548. Sua função era de controlar as capitanias e praticar as sesmarias. As terras, portanto, não possuíam valor financeiro, pois eram distribuídas e não vendidas.
No decorrer político da colonização brasileira, houve o período denominado de União Ibérica (1580 – 1640), em que a colônia ficou sob o controle da Espanha pelo problema na sucessão do trono português. Após o fim da União Ibérica, a coroa portuguesa expulsou os espanhóis e voltou a ter o monopólio da colônia.
O século XVIII na colônia foi marcado por uma forte agitação nas descobertas das minas de ouro, gerando grande entusiasmo por parte dos portugueses. Várias cidades do Brasil surgiram pela busca do ouro, entre elas estão Diamantina e Ouro Preto. A colonização estendeu-se até meados do século XIX, quando em Sete de Setembro de 1822, o Brasil conseguiu sua independência, inaugurando o Período Imperial."
"""

In [9]:
messages=[{"role":"system","content": system_context_pt},
          {"role":"user","content": prompt_obj_pt + texto}]
response_pt = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt}]
response_stud_pt = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt + add_pt + texto}]
response_stud_pt_text = gpt.chat(messages)

print(response_pt)

Teste de História: Colonização do Brasil

Pergunta 1: Qual foi o ano em que os navios portugueses ancoraram no litoral brasileiro, dando início à colonização do Brasil?
Pergunta 2: O que foi o Tratado de Tordesilhas e qual era o seu objetivo?
Pergunta 3: O que eram as capitanias hereditárias e qual era o objetivo de sua criação?
Pergunta 4: Quem foi o primeiro governador-geral do Brasil e qual era sua função?
Pergunta 5: Qual foi o período em que a colônia brasileira ficou sob o controle da Espanha?
Pergunta 6: O que marcou o século XVIII na colônia brasileira?
Pergunta 7: Cite duas cidades brasileiras que surgiram devido à busca por ouro durante o período colonial.
Pergunta 8: O que aconteceu em Sete de Setembro de 1822 no Brasil?
Pergunta 9: Qual foi o período correspondente aos trinta primeiros anos das expedições portuguesas no Brasil?
Pergunta 10: Quem liderou a expedição que oficializou a colonização brasileira em 1530?


In [10]:
print(response_stud_pt)

Teste de História: Colonização do Brasil

Pergunta 1: Qual foi o ano em que os navios portugueses ancoraram no litoral brasileiro, dando início à colonização do Brasil?
Resposta: 1500.

Pergunta 2: O que foi o Tratado de Tordesilhas e qual era o seu objetivo?
Resposta: Foi um acordo entre Portugal e Espanha em 1494 para dividir as terras "descobertas e por descobrir" fora da Europa, estabelecendo um meridiano a oeste do qual pertenceria a Espanha e a leste a Portugal.

Pergunta 3: O que eram as capitanias hereditárias e qual era o objetivo de sua criação?
Resposta: Divisões administrativas do território brasileiro do período colonial, com o objetivo de facilitar a colonização e exploração das terras.

Pergunta 4: Quem foi o primeiro governador-geral do Brasil e qual era sua função?
Resposta: Tomé de Sousa, cuja função era administrar a colônia em nome de Portugal.

Pergunta 5: Qual foi o período em que a colônia brasileira ficou sob o controle da Espanha?
Resposta: União Ibérica, de 15

In [11]:
print(response_stud_pt_text)

Teste de História: Colonização do Brasil

Pergunta 1: Qual foi o ano em que os navios portugueses ancoraram no litoral brasileiro, dando início à colonização do Brasil?
Sua resposta: 1500.

Pergunta 2: O que foi o Tratado de Tordesilhas e qual era o seu objetivo?
Sua resposta: O Tratado de Tordesilhas foi um acordo entre Portugal e Espanha para dividir as terras descobertas no Novo Mundo. Seu objetivo era estabelecer uma linha imaginária que determinava quais terras pertenceriam a cada país.

Pergunta 3: O que eram as capitanias hereditárias e qual era o objetivo de sua criação?
Sua resposta: As capitanias hereditárias eram grandes faixas de terra concedidas a capitães donatários com o objetivo de colonizar e explorar economicamente o território brasileiro.

Pergunta 4: Quem foi o primeiro governador-geral do Brasil e qual era sua função?
Sua resposta: O primeiro governador-geral do Brasil foi Tomé de Souza, e sua função era controlar as capitanias e administrar a colônia em nome da co

In [12]:
messages=[{"role":"system","content": system_context_pt},
          {"role":"user","content": prompt_sub_pt + texto}]
response_pt = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt}]
response_stud_pt = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt + add_pt + texto}]
response_stud_pt_text = gpt.chat(messages) 

print(response_pt)

Teste sobre a Colonização do Brasil

Pergunta 1: Qual foi a data em que os navios portugueses ancoraram no litoral brasileiro, dando início à colonização do Brasil?
Pergunta 2: O que foi o Tratado de Tordesilhas e qual era o objetivo desse acordo?
Pergunta 3: O que eram as capitanias hereditárias e qual era o objetivo de sua criação?
Pergunta 4: Quem foi o primeiro governador-geral do Brasil e qual era sua função?
Pergunta 5: O que eram as sesmarias e como eram praticadas na colônia brasileira?
Pergunta 6: O que foi a União Ibérica e qual foi sua influência na colonização brasileira?
Pergunta 7: Qual foi o impacto da descoberta das minas de ouro no século XVIII na colônia brasileira?
Pergunta 8: Cite duas cidades brasileiras que surgiram devido à busca pelo ouro.
Pergunta 9: Até quando se estendeu o período de colonização do Brasil?
Pergunta 10: Qual foi a data em que o Brasil conquistou sua independência?


In [13]:
print(response_stud_pt)

Teste sobre a Colonização do Brasil

Pergunta 1: Qual foi a data em que os navios portugueses ancoraram no litoral brasileiro, dando início à colonização do Brasil?
Sua resposta: 22 de abril de 1500.

Pergunta 2: O que foi o Tratado de Tordesilhas e qual era o objetivo desse acordo?
Sua resposta: O Tratado de Tordesilhas foi um acordo entre Portugal e Espanha em 1494 para dividir as terras "descobertas e por descobrir" fora da Europa, estabelecendo um meridiano a oeste do qual pertenceria a Espanha e a leste, a Portugal.

Pergunta 3: O que eram as capitanias hereditárias e qual era o objetivo de sua criação?
Sua resposta: As capitanias hereditárias eram territórios doados pela Coroa portuguesa a donatários com o objetivo de colonizar e administrar regiões do Brasil.

Pergunta 4: Quem foi o primeiro governador-geral do Brasil e qual era sua função?
Sua resposta: Tomé de Sousa foi o primeiro governador-geral do Brasil, sua função era administrar a colônia em nome da Coroa portuguesa.

Pe

In [14]:
print(response_stud_pt_text)

Teste sobre a Colonização do Brasil

Pergunta 1: Qual foi a data em que os navios portugueses ancoraram no litoral brasileiro, dando início à colonização do Brasil?
Sua resposta: Em 1500.

Pergunta 2: O que foi o Tratado de Tordesilhas e qual era o objetivo desse acordo?
Sua resposta: O Tratado de Tordesilhas foi um acordo entre Portugal e Espanha para dividir as terras descobertas no Novo Mundo. O objetivo era evitar conflitos sobre as áreas de exploração.

Pergunta 3: O que eram as capitanias hereditárias e qual era o objetivo de sua criação?
Sua resposta: As capitanias hereditárias eram grandes faixas de terra concedidas a nobres para colonizar e explorar o território brasileiro em nome de Portugal.

Pergunta 4: Quem foi o primeiro governador-geral do Brasil e qual era sua função?
Sua resposta: O primeiro governador-geral do Brasil foi Tomé de Souza, e sua função era centralizar o poder e administrar a colônia em nome de Portugal.

Pergunta 5: O que eram as sesmarias e como eram pra

In [15]:
texto = """
A Revolta da Chibata foi organizada pelos marinheiros brasileiros que estavam em navios atracados na Baía de Guanabara, no Rio de Janeiro. Esse motim dos marinheiros aconteceu entre os dias 22 e 27 de novembro de 1910 e teve como principal razão a insatisfação dos marinheiros com os castigos físicos a que eram sujeitos: as chibatadas.
A Revolta da Chibata ficou extremamente conhecida por ter sido uma reação dos marinheiros brasileiros aos castigos físicos a que eram sujeitos. A Marinha brasileira tinha como prática na época o uso da chibatada como forma de punição de seus marinheiros caso violassem o código de conduta da corporação.
O uso da chibatada como punição pela Marinha brasileira era uma herança colonial transmitida pela Marinha portuguesa a partir de um código chamado Artigos de Guerra. O uso das chibatadas como punição era dedicado aos marinheiros que ocupavam uma posição mais baixa na hierarquia da Marinha. Em geral, os cargos mais baixos da Marinha eram ocupados por negros e mestiços.
A insatisfação dos marinheiros com as chibatadas existia já havia um tempo, inclusive, pouco antes do motim, os marinheiros haviam manifestado a sua insatisfação com a punição de um marinheiro quando navegavam nas proximidades da costa do Chile. O estopim que levou ao início do movimento ocorreu com a punição dada para Marcelino Rodrigues Menezes – punido com 250 chibatadas.
A Revolta da Chibata, no entanto, não aconteceu somente pela insatisfação com os castigos físicos, mas manifestou a insatisfação dos marinheiros, todos pertencentes às classes baixas, com o racismo existente na corporação e com a desigualdade da sociedade. Para as historiadoras Lilia Schwarcz e Heloisa Starling, a Revolta da Chibata “também se encarregou de expor publicamente a violência do Estado contra a população pobre, o racismo e a crueldade reinante em meio à oficialidade das Forças Armadas”
Foi no dia 22 de novembro de 1910 que os marinheiros rebelaram-se e tomaram o controle de diferentes encouraçados da Marinha: Minas Gerais, São Paulo, Bahia, além de um navio-patrulha chamado Deodoro. Os marinheiros tomaram o controle das embarcações e afirmavam que, se as autoridades não decretassem o fim dos castigos físicos, a cidade do Rio de Janeiro seria bombardeada.
Coincidentemente, a Revolta da Chibata iniciou-se exatamente no dia que o novo presidente brasileiro – Hermes da Fonseca – realizava uma festa em comemoração a sua posse como presidente do Brasil. O manifesto que relatava a insatisfação dos marinheiros foi enviado ao presidente brasileiro e, provavelmente, pode ter sido escrito por Adalberto Ferreira Ribas. O líder dos marujos revoltosos foi João Cândido, que recebeu a alcunha de Almirante Negro.
O manifesto dos marinheiros revoltosos foi considerado um documento muito bem escrito e nele os marinheiros faziam uma série de reivindicações:
substituição de oficiais, aumento do soldo, fim dos castigos físicos, melhor tratamento na Marinha brasileira etc.
O governo brasileiro aceitou as condições dos marinheiros e prometeu-lhes anistia caso colocassem fim à revolta.
Assim, os marinheiros revoltosos entregaram as embarcações para seus oficiais no dia 26 de novembro de 1910. No entanto, a promessa feita pelo governo brasileiro não foi mantida e, no dia 4 de dezembro de 1910, veio a resposta do governo aos revoltosos. Ao todo, o governo brasileiro prendeu 22 marujos e enviou-os para a Ilha das Cobras, local onde foram aprisionados e torturados.
Além dos marinheiros que foram presos e torturados na Ilha das Cobras, outros marujos foram fuzilados e muitos foram enviados para o Acre, onde foram obrigados a trabalhar nos seringais da região.
"""

In [16]:
messages=[{"role":"system","content": system_context_pt},
          {"role":"user","content": prompt_obj_pt + texto}]
response_pt = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt}]
response_stud_pt = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt + add_pt + texto}]
response_stud_pt_text = gpt.chat(messages) 

print(response_pt)

Teste: A Revolta da Chibata

Pergunta 1: O que motivou a Revolta da Chibata entre os marinheiros brasileiros em 1910?
Pergunta 2: Qual era a prática da Marinha brasileira que gerou insatisfação entre os marinheiros?
Pergunta 3: Qual era a herança colonial relacionada ao uso da chibata como punição na Marinha brasileira?
Pergunta 4: Quem ocupava em sua maioria os cargos mais baixos na hierarquia da Marinha e eram mais frequentemente punidos com chibatadas?
Pergunta 5: O que desencadeou o início da Revolta da Chibata em 1910?
Pergunta 6: Quais embarcações foram tomadas pelos marinheiros revoltosos durante a Revolta da Chibata?
Pergunta 7: Quem foi o líder dos marujos revoltosos durante a Revolta da Chibata e qual alcunha ele recebeu?
Pergunta 8: Quais eram as reivindicações feitas pelos marinheiros no manifesto enviado ao presidente brasileiro durante a Revolta da Chibata?
Pergunta 9: O que o governo brasileiro prometeu aos marinheiros revoltosos em troca do fim da revolta?
Pergunta 10: 

In [17]:
print(response_stud_pt)

Teste: A Revolta da Chibata

Pergunta 1: O que motivou a Revolta da Chibata entre os marinheiros brasileiros em 1910?
Sua resposta: A principal motivação foi a insatisfação com os castigos físicos, especialmente as chibatadas, aplicados pela Marinha.

Pergunta 2: Qual era a prática da Marinha brasileira que gerou insatisfação entre os marinheiros?
Sua resposta: A prática de aplicar chibatadas como forma de punição.

Pergunta 3: Qual era a herança colonial relacionada ao uso da chibata como punição na Marinha brasileira?
Sua resposta: A herança colonial estava ligada à prática de punições físicas severas, como a chibata, herdada do período escravocrata.

Pergunta 4: Quem ocupava em sua maioria os cargos mais baixos na hierarquia da Marinha e eram mais frequentemente punidos com chibatadas?
Sua resposta: Os marinheiros negros e mulatos, que compunham a maioria dos cargos mais baixos na hierarquia da Marinha, eram os mais punidos com chibatadas.

Pergunta 5: O que desencadeou o início da 

In [18]:
print(response_stud_pt_text)

Teste: A Revolta da Chibata

Pergunta 1: O que motivou a Revolta da Chibata entre os marinheiros brasileiros em 1910?
Sua resposta: A insatisfação dos marinheiros com os castigos físicos, especialmente as chibatadas, impostos pela Marinha brasileira.

Pergunta 2: Qual era a prática da Marinha brasileira que gerou insatisfação entre os marinheiros?
Sua resposta: O uso da chibata como forma de punição para os marinheiros que violassem o código de conduta da corporação.

Pergunta 3: Qual era a herança colonial relacionada ao uso da chibata como punição na Marinha brasileira?
Sua resposta: A prática da chibata como punição era uma herança colonial transmitida pela Marinha portuguesa através do código chamado Artigos de Guerra.

Pergunta 4: Quem ocupava em sua maioria os cargos mais baixos na hierarquia da Marinha e eram mais frequentemente punidos com chibatadas?
Sua resposta: Os marinheiros negros e mestiços, que ocupavam os cargos mais baixos na hierarquia da Marinha, eram os mais freque

In [19]:
messages=[{"role":"system","content": system_context_pt},
          {"role":"user","content": prompt_sub_pt + texto}]
response_pt = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt}]
response_stud_pt = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt + add_pt + texto}]
response_stud_pt_text = gpt.chat(messages) 

print(response_pt)

**Teste sobre a Revolta da Chibata**

Pergunta 1: Qual foi a principal razão que levou os marinheiros brasileiros a se rebelarem durante a Revolta da Chibata?

Pergunta 2: Qual era a prática da Marinha brasileira que gerava insatisfação entre os marinheiros?

Pergunta 3: De acordo com o texto, qual era a herança colonial transmitida pela Marinha portuguesa que envolvia o uso da chibata como punição?

Pergunta 4: Quem ocupava, em sua maioria, os cargos mais baixos na hierarquia da Marinha e eram os principais alvos das chibatadas?

Pergunta 5: O que aconteceu pouco antes do motim que demonstrou a insatisfação dos marinheiros com as punições físicas?

Pergunta 6: Quem foi o líder dos marujos revoltosos durante a Revolta da Chibata?

Pergunta 7: Qual era o nome dado ao líder dos marujos revoltosos?

Pergunta 8: Quais eram algumas das reivindicações feitas pelos marinheiros no manifesto enviado ao presidente brasileiro?

Pergunta 9: O que o governo brasileiro prometeu aos marinheiros revol

In [20]:
print(response_stud_pt)

**Teste sobre a Revolta da Chibata**

Pergunta 1: Qual foi a principal razão que levou os marinheiros brasileiros a se rebelarem durante a Revolta da Chibata?
Sua resposta: A principal razão foi a prática de punições físicas, como chibatadas, aplicadas pela Marinha.

Pergunta 2: Qual era a prática da Marinha brasileira que gerava insatisfação entre os marinheiros?
Sua resposta: A prática de punições físicas, especialmente o uso da chibata.

Pergunta 3: De acordo com o texto, qual era a herança colonial transmitida pela Marinha portuguesa que envolvia o uso da chibata como punição?
Sua resposta: A herança colonial era a punição física como forma de disciplina, comum na Marinha portuguesa.

Pergunta 4: Quem ocupava, em sua maioria, os cargos mais baixos na hierarquia da Marinha e eram os principais alvos das chibatadas?
Sua resposta: Os marinheiros negros e mulatos, que ocupavam os cargos mais baixos na hierarquia, eram os principais alvos das chibatadas.

Pergunta 5: O que aconteceu pou

In [21]:
print(response_stud_pt_text)

**Teste sobre a Revolta da Chibata**

Pergunta 1: Qual foi a principal razão que levou os marinheiros brasileiros a se rebelarem durante a Revolta da Chibata?
Sua resposta: A insatisfação com os castigos físicos, especialmente as chibatadas.

Pergunta 2: Qual era a prática da Marinha brasileira que gerava insatisfação entre os marinheiros?
Sua resposta: O uso da chibata como forma de punição.

Pergunta 3: De acordo com o texto, qual era a herança colonial transmitida pela Marinha portuguesa que envolvia o uso da chibata como punição?
Sua resposta: O uso das chibatadas como punição transmitido pelos Artigos de Guerra.

Pergunta 4: Quem ocupava, em sua maioria, os cargos mais baixos na hierarquia da Marinha e eram os principais alvos das chibatadas?
Sua resposta: Negros e mestiços, marinheiros das classes baixas.

Pergunta 5: O que aconteceu pouco antes do motim que demonstrou a insatisfação dos marinheiros com as punições físicas?
Sua resposta: A manifestação de insatisfação com a puniç

In [22]:
texto = """
Genetics Overview for High School Students (Resumo de Genética para Estudantes do Ensino Médio)
Genetics is the study of how traits are passed from one generation to another. It plays a crucial role in understanding how living organisms inherit and express characteristics. Here are some key concepts in genetics:
Heredity: Heredity is the process by which characteristics are passed from parents to offspring. This is accomplished through genes, which are segments of DNA.
DNA: DNA (Deoxyribonucleic Acid) is the genetic material in all living organisms. It carries the instructions for building and maintaining an organism. DNA is organized into structures called chromosomes.
Genes: Genes are specific segments of DNA that code for particular traits or characteristics. For example, genes determine things like eye color, blood type, and susceptibility to certain diseases.
Alleles: Genes can have different forms, known as alleles. Alleles can be dominant or recessive, and they influence how traits are expressed.
Punnett Squares: Punnett squares are a tool used to predict the possible genetic outcomes of a mating between two individuals. They show the probability of different combinations of alleles in their offspring.
Mendelian Inheritance: Gregor Mendel, an Austrian scientist, is known for his work on pea plants, which laid the foundation for our understanding of inheritance. His principles, such as the Law of Segregation and the Law of Independent Assortment, explain how alleles are passed on.
Genetic Disorders: Some genetic variations can lead to genetic disorders. These conditions can be caused by mutations in genes and may result in inherited diseases or conditions.
Modern Genetics: Today, genetics has advanced significantly with the discovery of the structure of DNA, the sequencing of the human genome, and advancements in genetic engineering and biotechnology.
"""

In [23]:
messages=[{"role":"system","content": system_context_en},
          {"role":"user","content": prompt_obj_en + texto}]
response_en = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en}]
response_stud_en = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en + add_en + texto}]
response_stud_en_text = gpt.chat(messages) 

print(response_en)

Genetics Test for High School Students

Question 1: Define heredity and explain its significance in genetics.
Question 2: What is DNA and what role does it play in living organisms?
Question 3: Describe the relationship between genes and traits in organisms.
Question 4: What are alleles and how do they influence the expression of traits?
Question 5: How are Punnett squares used in genetics, and what do they help predict?
Question 6: Who is Gregor Mendel and what are some of his contributions to the field of genetics?
Question 7: Explain the difference between dominant and recessive alleles with an example.
Question 8: What are genetic disorders and how do they relate to mutations in genes?
Question 9: Discuss the significance of the discovery of the structure of DNA in modern genetics.
Question 10: How has genetic engineering and biotechnology advanced the field of genetics in recent years?


In [24]:
messages=[{"role":"system","content": system_context_en},
          {"role":"user","content": prompt_sub_en + texto}]
response_en = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en}]
response_stud_en = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en + add_en + texto}]
response_stud_en_text = gpt.chat(messages) 

print(response_en)

Genetics Test for High School Students

Question 1: Define heredity and explain its significance in genetics.
Question 2: What is DNA, and why is it important in living organisms?
Question 3: Describe the role of genes in determining traits in organisms.
Question 4: What are alleles, and how do they influence the expression of traits?
Question 5: Explain how Punnett squares are used in predicting genetic outcomes.
Question 6: Who is Gregor Mendel, and what are his contributions to the field of genetics?
Question 7: Discuss the concept of Mendelian inheritance and provide examples.
Question 8: What are genetic disorders, and how do they relate to mutations in genes?
Question 9: How has modern genetics advanced compared to historical understandings of genetics?
Question 10: Provide an example of a genetic engineering application in modern genetics.


In [25]:
texto = """
When the London Company sent out its first expedition to begin colonizing Virginia on December 20, 1606, it was by no means the first European attempt to exploit North America. In 1564, for example, French Protestants (Huguenots) built a colony near what is now Jacksonville, Florida. This intrusion did not go unnoticed by the Spanish, who had previously claimed the region. The next year, the Spanish established a military post at St. Augustine; Spanish troops soon wiped out the French interlopers residing but 40 miles away.
Meanwhile, Basque, English, and French fishing fleets became regular visitors to the coasts from Newfoundland to Cape Cod. Some of these fishing fleets even set up semi-permanent camps on the coasts to dry their catches and to trade with local people, exchanging furs for manufactured goods. For the next two decades, Europeans' presence in North America was limited to these semi-permanent incursions. Then in the 1580s, the English tried to plant a permanent colony on Roanoke Island (on the outer banks of present-day North Carolina), but their effort was short-lived.
In the early 1600s, in rapid succession, the English began a colony (Jamestown) in Chesapeake Bay in 1607, the French built Quebec in 1608, and the Dutch began their interest in the region that became present-day New York. Within another generation, the Plymouth Company (1620), the Massachusetts Bay Company (1629), the Company of New France (1627), and the Dutch West India Company (1621) began to send thousands of colonists, including families, to North America. Successful colonization was not inevitable. Rather, interest in North America was a halting, yet global, contest among European powers to exploit these lands.
There is another very important point to keep in mind:  European colonization and settlement of North America (and other areas of the so-called "new world") was an invasion of territory controlled and settled for centuries by Native Americans. To be sure, Native American control and settlement of that land looked different to European eyes. Nonetheless, Native American groups perceived the Europeans' arrival as an encroachment and they pursued any number of avenues to deal with that invasion. That the Native American were unsuccessful in the long run in resisting or in establishing a more favorable accommodation with the Europeans was as much the result of the impact of European diseases as superior force of arms. Moreover, to view the situation from Native American perspectives is essential in understanding the complex interaction of these very different peoples.
Finally, it is also important to keep in mind that yet a third group of people--in this case Africans--played an active role in the European invasion (or colonization) of the western hemisphere. From the very beginning, Europeans' attempts to establish colonies in the western hemisphere foundered on the lack of laborers to do the hard work of colony-building. The Spanish, for example, enslaved the Native American in regions under their control. The English struck upon the idea of indentured servitude to solve the labor problem in Virginia. Virtually all the European powers eventually turned to African slavery to provide labor on their islands in the West Indies. Slavery was eventually transferred to other colonies in both South and North America.
Because of the interactions of these very diverse peoples, the process of European colonization of the western hemisphere was a complex one, indeed. Individual members of each group confronted situations that were most often not of their own making or choosing. These individuals responded with the means available to them. For most, these means were not sufficient to prevail. Yet these people were not simply victims; they were active agents trying to shape their own destinies. That many of them failed should not detract from their efforts.
"""

In [26]:
messages=[{"role":"system","content": system_context_en},
          {"role":"user","content": prompt_sub_en + texto}]
response_en = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en}]
response_stud_en = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en + add_en + texto}]
response_stud_en_text = gpt.chat(messages) 

print(response_en)

Test Title: European Colonization of North America

Question 1: When did the London Company send out its first expedition to colonize Virginia?
Question 2: What European group built a colony near present-day Jacksonville, Florida in 1564?
Question 3: Which European power established a military post at St. Augustine in response to the French colony in Florida?
Question 4: What did Basque, English, and French fishing fleets do along the coasts of North America in the late 16th century?
Question 5: Where did the English attempt to plant a permanent colony in the 1580s that was short-lived?
Question 6: Name three European colonies established in North America in the early 1600s.
Question 7: What important point should be kept in mind regarding European colonization of North America in relation to Native Americans?
Question 8: How did Native American groups perceive the arrival of Europeans in North America?
Question 9: What labor solution did the English use in Virginia to address the lack

In [27]:
messages=[{"role":"system","content": system_context_en},
          {"role":"user","content": prompt_obj_en + texto}]
response_en = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en}]
response_stud_en = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_en},
          {"role":"user","content": prompt_stud_en + response_en + add_en + texto}]
response_stud_en_text = gpt.chat(messages) 

print(response_en)

Test Title: European Colonization of North America

Question 1: When did the London Company send out its first expedition to colonize Virginia?
Question 2: Which European group built a colony near present-day Jacksonville, Florida in 1564?
Question 3: What did the Spanish establish in response to the French colony near Jacksonville?
Question 4: Which English colony was attempted on Roanoke Island in the 1580s?
Question 5: Name the colony established by the English in Chesapeake Bay in 1607.
Question 6: In what year did the French build Quebec?
Question 7: Which European power began their interest in the region that became present-day New York in the early 1600s?
Question 8: Name one of the companies that began sending colonists to North America within a generation of the early settlements.
Question 9: What was a significant factor that contributed to the Native Americans' inability to resist European colonization in the long run?
Question 10: How did the English solve the labor problem

In [28]:
texto ="""
Marielle Franco foi assassinada do Rio de Janeiro no dia 14 de março de 2018. Há cinco anos o movimento negro e seus ativistas, os grupos de esquerda, a família Franco e todos e todas que verdadeiramente querem uma sociedade democrática e livre querem saber quem foram os mandantes do covarde e bárbaro extermínio de uma mulher negra, lésbica, socialista, vereadora de esquerda (PSol) e formada nos morros cariocas.
Se precisamos ser cuidadosos em certos textos e escrito – não é o caso aqui. Pois na medida em que os executores do crime foram dois policiais militares (que circunstancialmente eram aposentados), é mais do que evidente que seus mandantes são agentes do Estado: de uma maneira ou de outra, direta ou indiretamente, ou influenciando ou organicamente. Esses ainda não sabemos quem são.
Há uma passagem no livro Dez dias que abalaram o mundo, de John Reed, a mais notável narrativa da Revolução Russa de 1917 até hoje escrita, em que presenciamos a seguinte cena – “os pobres dos asilos estavam quase nus. Delegações de inválidos, caindo de fome, órfãos, com as faces encovadas e lívidas, assaltaram o edifício [da Assistência Pública]. [Aleksandra] Kolontai, com os olhos rasos de água foi obrigada a mandar deter […] [os burocratas] para obrigá-los a entregar as chaves das salas e dos cofres”.[1]
Nas “salas” e nos “cofres” estavam muitos dos segredos do Estado czarista-burguês, e a classe dominante russa fez de tudo para mantê-los assim, sem que o povo tivesse conhecimento de suas jogadas egoístas e autointeressadas que lançava a maioria esmagadora da população na mais profunda miséria. É evidente que estamos distantes, de um evento como o de 1917; e não queremos que até lá os mandantes do assassinato de Marielle fiquem ocultos nas salas e nos cofres de nosso Estado e das instituições públicas que o constitui.
Quando Marielle foi assassinada o Brasil passava por um dos momentos do que se pode chamar de sua “contrarrevolução”. Em 2008 o mundo enfrentou uma das maiores crises do sistema capitalista internacional. A eclosão dos subprimes acompanhada com a queda do Lehamnn Brothers nos Estados Unidos teve efeito em cadeia por toda a economia internacional. Quando a crise se abateu na vida material dos trabalhadores e trabalhadoras, diversos movimentos contra-hegemônicos demonstraram as possibilidades (e a necessidade) de construirmos outra sociedade: Occupy Wall Street, Primavera Árabe, Syriza, Indignados e Podemos tomaram as ruas em suas respectivas sociedades deixando um rastro de fogo e esperança nos corações e mentes daqueles que querem uma humanidade livre do jugo do capital.
Na medida em que as respostas ao esfacelamento das finanças mundiais não foram à altura do esperado pelos mais afetados, houve um processo de rápido desgaste de boa parte dos movimentos que se apresentaram como alternativa opção política. As novas sociais-democracias, nos termos de Susan Watkins,[2] enredadas pelas técnicas de governo das democracias liberais-representativas – essas que já vinham atravessando momentos de contestação por não realizarem suas promessas (Norberto Bobbio) – comprometeram não só a si enquanto atores e sujeitos sociais da suposta transformação desejada, a própria democracia perderia o que lhe restava de confiança.
Com o passar dos anos já não era vista como o único e exclusivo regime político. Dado a justaposição a esse cenário da Troika (Comissão Europeia, o Banco Central Europeu (BCE) e o Fundo Monetário Internacional (FMI)), vale dizer, da reorganização de instituições internacionais de decisão no continente europeu o esvaziamento intencional e “planejado” do que restava de democracia teria efeitos devastadores. (Nos Estados Unidos, Barack Obama que governava para recuperar o sistema financeiro, afagar celebridades negras e capturar e/ou matar Osama Bin Laden.)
A combinação dialético-imanente dessas circunstâncias resultaria na ascensão do que chamamos hoje de a nova direita. Trump, Erdogan, Salvini, Johson, Orban, Duterte e Bolsonaro apareceram no horizonte político imediato. Eles foram a luva de aço do novo regime político e de acumulação do neoliberalismo; que Verônica Gago chama de acumulação por extração[3] – violência extrema contra os/as que trabalham, financeirização absoluta das formas cotidianas de vida, devastação da natureza e psique economicista-eficiente. As relações sociais necessárias exigiam além disso, racismo, machismo, homofobia assassina, xenofobia e fascistização da vida.
A desigualdade combinada da luta de classes fez com que esse quadro de referência histórica chegasse no Brasil em junho de 2013. Manifestações cobriram todo o país e um levante insurrecional, talvez, nunca presenciado por aqui. Junho de 2013 é o principal evento das últimas décadas na sociedade brasileira sem qualquer sombra de dúvida: nele expressou-se a realidade de uma transformação radical do Brasil. O que se seguiu a ele, de certa maneira, foi o que se seguiu na Europa e nos Estados Unidos – uma contrarrevolução preventiva.
As forças de direita se restabeleceram na fresta que se abriu; o balanço abrangente de 2013 ainda está para ser feito pela esquerda, seus e suas intelectuais, pesquisadores e pesquisadoras críticos. As fases da contrarrevolução brasileira podem ser, esquematicamente, estabelecidas da seguinte maneira: 2014 (desestabilização da eleição pela contestação do PSDB à vitória de Dilma Rousseff do PT); 2015 (intensificação das mobilizações da direita exigindo a deposição de Dilma organizadas por MBL, Vem pra Rua, Revoltados On-Line, sustentados por aparelhos privados ideológicos – Millenium, Institutos Liberal e Von Mises e toda a imprensa corporativa e financiadas por grupos econômicos interessados); 2016 (golpe contra a presidenta eleita em 2014, surgimento de Jair Bolsonaro anunciando a “volta” katecontica de Ustra e sua substituição por Michel Temer); 2017 (o programa econômico ponte-para-o-futuro que reposicionava o capitalismo brasileiro no “novo” regime de acumulação neoliberal que dissemos acima); 2018 (a ocupação do Rio de Janeiro, via GLO- Garantia da Lei da Ordem, dos militares sob o comando de Braga Neto).
É nesse contexto e/ou conjuntura que Marielle Franco foi assassinada a mando e cruelmente por Ronnie Lessa e Elcio Vieira de Queiroz – o primeiro havia sido membro competente e eficaz, segundo Rodrigo Rodrigues Pimentel, das forças de elite BOPE-Batalhão de Operações Especiais da polícia militar do Rio de Janeiro e o segundo ex-sargento, também, da polícia militar carioca. Os detalhes do crime contra Marielle são dos mais sórdidos e nefastos – típico da elite branca dominante brasileira desde os dias da colônia em que praticavam, verdadeira caça, prisão e castigos infinitos contra os corpos negros que fugia em busca da liberdade e contestavam aquela ordem escravagista de então. Mas agora com toda a técnica e tecnologia disponíveis.
Segundo consta Marielle foi vigiada por mais de dois meses por Lessa e Queiroz, sua rotina estudada com precisão, os lugares que frequentava enquanto militante do seu povo, os momentos de maior oportunidade e facilitação para efetivar a emboscada covarde; as armas agora não são mais o chicote no açoite, o tronco e a forquilha na garganta – mas pistolas de alto calibre e de alto impacto; a perseguição não mais a cavalo e sim com veículos de última geração e a organização efetiva do extermínio “racionalmente” – segundo investigações Ronnie Lessa e Elcio Queiroz espreitaram Marielle durante a noite do crime como sicários brancos que são – elaborada para o dia de 14 de março de 2018.
O erro foi acreditar que fazendo desaparecer Marielle fariam desaparecer sua luta histórica e política. É a sempre presente arrogância e esnobismo dos de cima. Se esquecem que não é a primeira vez que figuras que luta autenticamente pelos de baixo – ativistas, militantes, intelectuais, políticos, revolucionários e revolucionárias – tem suas vidas retiradas pelas forças da ordem vigente. Foi assim com os e as líderes Quilombolas; com os resistentes de Canudos; na Comuna de Paris; na Argélia de Frantz Fanon, na Espanha de 1936, com Rosa Luxemburgo, com as personalidades destacadas dos Panteras Negras – e foi assim com Marielle Franco.
E desde então florescem novos movimentos, novos indivíduos para falar pelos que nada tem a não ser o suor do rosto, novas figuras radicais e destemidas, novos grupos e coletivos dos que resistem às agruras do modo de produção burguês. Em 14 de março de 2023 completam-se cinco anos do assassinato de nossa Marielle; não descaçaremos até que os poltrões, que a mandaram matar para que ela não fosse um obstáculo para seus interesses materiais, sejam revelados – até lá, e até que a sociedade que Marielle desejou para os seus e para as suas irrompe na história, ela mesma será a alegoria incandescente na busca incansável para o fim da exploração, opressão e violência contra os subalternos e subalternas e que a retirou de entre nós.
"""

texto = """
O segundo kit de joias enviado pelo governo da Arábia Saudita, e que entrou ilegalmente no Brasil, foi entregue em mãos à então primeira dama, Michelle Bolsonaro (PL), e conferido pessoalmente pelo marido dela e então presidente, Jair Bolsonaro (PL), em 29 de novembro de 2022. A informação foi publicada originalmente pelo jornal O Estado de S. Paulo, que teve acesso a depoimento de uma servidora que trabalhava no Palácio do Planalto na época.

Segundo o depoimento, prestado à Polícia Federal (PF) no contexto das investigações sobre o caso, as joias foram recebidas por Michelle no Palácio da Alvorada, residência oficial da presidência. A servidora atuava no Gabinete Adjunto de Documentação Histórica (GADH), que presta serviços ao Gabinete Pessoal da Presidência e tem a atribuição de receber e fazer a triagem de presentes oficiais.

O conjunto em questão entrou no país em outubro de 2021, trazido pela comitiva do então minstro de Minas e Energia, Bento Albuquerque, e conta com cinco itens: um relógio, um anel, um par de abotoaduras, um rosário e uma caneta, todos em ouro com diamantes. Os valores dos objetos, quando somados, passam de R$ 1 milhão.
"""

In [29]:
messages=[{"role":"system","content": system_context_pt},
          {"role":"user","content": prompt_sub_pt + texto}]
response_pt = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt}]
response_stud_pt = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt + add_pt + texto}]
response_stud_pt_text = gpt.chat(messages) 

print(response_pt)

Teste: Presente de Joias para Michelle Bolsonaro

Pergunta 1: Quem entregou o segundo kit de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Pergunta 2: Onde as joias foram entregues em mãos para Michelle Bolsonaro?
Pergunta 3: Quem conferiu pessoalmente as joias entregues para Michelle Bolsonaro?
Pergunta 4: Qual foi a data em que as joias foram entregues para Michelle Bolsonaro?
Pergunta 5: Onde a servidora que trabalhava no Palácio do Planalto na época prestou depoimento sobre o caso das joias?
Pergunta 6: Qual era a função da servidora que atuava no Gabinete Adjunto de Documentação Histórica (GADH)?
Pergunta 7: Quais eram os cinco itens que compunham o conjunto de joias enviado pela Arábia Saudita?
Pergunta 8: Quem trouxe o conjunto de joias para o Brasil?
Pergunta 9: Qual é o material das joias enviadas pela Arábia Saudita?
Pergunta 10: Qual é o valor total estimado dos objetos do conjunto de joias enviado para Michelle Bolsonaro?


In [30]:
print(response_stud_pt)

Teste: Presente de Joias para Michelle Bolsonaro

Pergunta 1: Quem entregou o segundo kit de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Sua resposta: O embaixador da Arábia Saudita no Brasil.

Pergunta 2: Onde as joias foram entregues em mãos para Michelle Bolsonaro?
Sua resposta: No Palácio do Planalto.

Pergunta 3: Quem conferiu pessoalmente as joias entregues para Michelle Bolsonaro?
Sua resposta: A própria Michelle Bolsonaro.

Pergunta 4: Qual foi a data em que as joias foram entregues para Michelle Bolsonaro?
Sua resposta: Em novembro de 2019.

Pergunta 5: Onde a servidora que trabalhava no Palácio do Planalto na época prestou depoimento sobre o caso das joias?
Sua resposta: Na Polícia Federal.

Pergunta 6: Qual era a função da servidora que atuava no Gabinete Adjunto de Documentação Histórica (GADH)?
Sua resposta: Ela era chefe de gabinete do GADH.

Pergunta 7: Quais eram os cinco itens que compunham o conjunto de joias enviado pela Arábia Saudita?
Sua 

In [31]:
print(response_stud_pt_text)

**Teste: Presente de Joias para Michelle Bolsonaro**

Pergunta 1: Quem entregou o segundo kit de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Resposta: O segundo kit de joias foi entregue em mãos à Michelle Bolsonaro pelo governo da Arábia Saudita.

Pergunta 2: Onde as joias foram entregues em mãos para Michelle Bolsonaro?
Resposta: As joias foram entregues em mãos para Michelle Bolsonaro no Palácio da Alvorada, residência oficial da presidência.

Pergunta 3: Quem conferiu pessoalmente as joias entregues para Michelle Bolsonaro?
Resposta: Jair Bolsonaro, então presidente, conferiu pessoalmente as joias entregues para Michelle Bolsonaro.

Pergunta 4: Qual foi a data em que as joias foram entregues para Michelle Bolsonaro?
Resposta: As joias foram entregues para Michelle Bolsonaro em 29 de novembro de 2022.

Pergunta 5: Onde a servidora que trabalhava no Palácio do Planalto na época prestou depoimento sobre o caso das joias?
Resposta: A servidora prestou depoimen

In [32]:
messages=[{"role":"system","content": system_context_pt},
          {"role":"user","content": prompt_obj_pt + texto}]
response_pt = gpt.chat(messages)

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt}]
response_stud_pt = gpt.chat(messages) 

messages=[{"role":"system","content": system_context_stud_pt},
          {"role":"user","content": prompt_stud_pt + response_pt + add_pt + texto}]
response_stud_pt_text = gpt.chat(messages) 

print(response_pt)

Teste: Presente de Joias para Michelle Bolsonaro

Pergunta 1: Quem entregou o segundo kit de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Pergunta 2: Onde as joias foram entregues em mãos para Michelle Bolsonaro?
Pergunta 3: Quem conferiu pessoalmente as joias entregues a Michelle Bolsonaro?
Pergunta 4: Quando as joias foram entregues a Michelle Bolsonaro?
Pergunta 5: Onde a servidora que trabalhava no Palácio do Planalto na época prestou depoimento sobre o caso das joias?
Pergunta 6: Qual era a função da servidora que prestou depoimento à Polícia Federal sobre o caso das joias?
Pergunta 7: Quantos itens compõem o conjunto de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Pergunta 8: Quais são os materiais principais das joias enviadas para Michelle Bolsonaro?
Pergunta 9: Quem trouxe o conjunto de joias para o Brasil?
Pergunta 10: Qual é o valor total estimado dos objetos de joias enviados para Michelle Bolsonaro?


In [33]:
print(response_stud_pt)

Teste: Presente de Joias para Michelle Bolsonaro

Pergunta 1: Quem entregou o segundo kit de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Sua resposta: Um diplomata saudita.

Pergunta 2: Onde as joias foram entregues em mãos para Michelle Bolsonaro?
Sua resposta: No Palácio do Alvorada.

Pergunta 3: Quem conferiu pessoalmente as joias entregues a Michelle Bolsonaro?
Sua resposta: A própria Michelle Bolsonaro.

Pergunta 4: Quando as joias foram entregues a Michelle Bolsonaro?
Sua resposta: Em dezembro de 2018.

Pergunta 5: Onde a servidora que trabalhava no Palácio do Planalto na época prestou depoimento sobre o caso das joias?
Sua resposta: Na Polícia Federal.

Pergunta 6: Qual era a função da servidora que prestou depoimento à Polícia Federal sobre o caso das joias?
Sua resposta: Secretária do cerimonial.

Pergunta 7: Quantos itens compõem o conjunto de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Sua resposta: 13 itens.

Pergunta 8: Q

In [34]:
print(response_stud_pt_text)


Pergunta 1: Quem entregou o segundo kit de joias enviado pelo governo da Arábia Saudita para Michelle Bolsonaro?
Resposta: O segundo kit de joias foi entregue pelo então ministro de Minas e Energia, Bento Albuquerque.

Pergunta 2: Onde as joias foram entregues em mãos para Michelle Bolsonaro?
Resposta: As joias foram entregues em mãos para Michelle Bolsonaro no Palácio da Alvorada, residência oficial da presidência.

Pergunta 3: Quem conferiu pessoalmente as joias entregues a Michelle Bolsonaro?
Resposta: As joias foram conferidas pessoalmente por Jair Bolsonaro, então presidente na época.

Pergunta 4: Quando as joias foram entregues a Michelle Bolsonaro?
Resposta: As joias foram entregues a Michelle Bolsonaro em 29 de novembro de 2022.

Pergunta 5: Onde a servidora que trabalhava no Palácio do Planalto na época prestou depoimento sobre o caso das joias?
Resposta: A servidora prestou depoimento à Polícia Federal.

Pergunta 6: Qual era a função da servidora que prestou depoimento à Pol